In [ ]:
import os
import pandas as pd

Let's find some viruses!  This notebook will go through the following steps:

1. Run three virus finders
2. Apply thresholds to filter out unlikely candidates
3. Merge results from virus finders

First, pick a SAG!

Check out the table [here](https://docs.google.com/spreadsheets/d/1yn6GsOv8dHwtsn2UKs5KIYUtCLsc1PL5Lsp6aMV6294/edit?usp=sharing) to select a SAG. Fill in your name next to your selected SAG.

Now assign your SAG ID to the SAG variable.  Let's also set some other variable names, such as the location of the working directory and where we will place the sag contigs.

In [ ]:
sag = '{SAG ID here}'

# setting a working subdirectory for virus finding
sagdir = "{}_vfinding".format(sag)

# setting a location to place SAG contigs
sag_contigs = os.path.join(sagdir, '{}_contigs.fasta'.format(sag))

Now let's create a working directory, and copy our SAG over to this directory

In [ ]:
!mkdir {sagdir}

!cp /mnt/storage/data/sag-data/gorg-dark/contigs/{sag}_contigs.fasta {sagdir}

#### VirSorter2

First let's make a directory for virsorter2 outputs:

In [ ]:
vs2_output_dir = os.path.join(sagdir, 'vs2')

!mkdir {vs2_output_dir}

VirSorter2 is installed within a conda environment.  To load virsorter2, open a terminal and enter:

```
source activate /mnt/storage/envs/vs2
```

The general command for virsorter2 is:

```
virsorter run -i {sag_contigs} -w {output_dir} -j {number of jobs}
```

For this tutorial, enter the below printed line into your terminal:

In [ ]:
print('virsorter run -i {sag_contigs} -w {vs2_output_dir} -j 2'.format(sag_contigs = sag_contigs, vs2_output_dir = vs2_output_dir))

#### VIBRANT

Next let's run this SAG through VIBRANT

In [ ]:
vib_outdir = os.path.join(sagdir, "vibrant")

!mkdir {vib_outdir}

We'll want to work in the VIBRANT conda environment to run this software in a terminal, rather than in this notebook. Load this environment in terminal by typing:

```
source activate /mnt/storage/envs/vibrant
```

The general command for vibrant is:
```
VIBRANT_run.py -i {contig} -t {cpus} -folder {outdirectory}
```

To run for your SAG, enter the printed output from the below line into your terminal:

In [ ]:
print('VIBRANT_run.py -i {sag_contigs} -t 2 -folder {vib_outdir}'.format(**locals()))

### DeepVirFinder

Next let's run our SAG through DeepVirFinder

In [ ]:
dvf_outdir = os.path.join(sagdir, "dvf")

!mkdir {dvf_outdir}

We'll want to work in the DeepVirFinder conda environment to run this software in a terminal, rather than in this notebook. Load this environment in terminal by typing:

```
source activate /mnt/storage/envs/dvf
```

The general command for vibrant is:
```
dvf.py -i {contig} -o {output directory} -c {cpus}
```

To run for your SAG, enter the printed output from the below line into your terminal:

In [ ]:
print('python /mnt/storage/envs/opt/DeepVirFinder/dvf.py -i {sag_contigs} -o {dvf_outdir} -c 2 '.format(**locals()))

Now let's explore the outputs!

### Virsorter2

Let's list the directory contents:

In [ ]:
!ls {vs2_output_dir}

The file we'll look at is called 'final-viral-score.tsv'.  Next, we'll load it into a pandas dataframe:

In [ ]:
vs2df = pd.read_csv(os.path.join(vs2_output_dir, "final-viral-score.tsv"), sep = "\t")
vs2df

Let's add/transform some columns so that they may be merged with other virus finder outputs:

In [ ]:
vs2df['contig'] = [i.split("|")[0] for i in vs2df['seqname']]
vs2df['vs2_type'] = [i.split("|")[-1] for i in vs2df['seqname']]
vs2df['sag'] = [i.split("_")[0] for i in vs2df['contig']]

VirSorter2 is one of the only workflows that will intentionally look for different types of viruses by running several different types of virus searches.  By using the default parameters, it searched for dsDNA phages and ssDNA viruses, but we could have asked it to search for additional types of viruses.

Your results may all look promising, but sometimes, this is not the case.  Let's apply a filter to this dataframe to only keep matches virsorter2 has high confidence in:

In pseudocode:
```
Keep the contig as a viral candidate if max_score > 0.9 
and
either
max_score_group == 'ssDNA'
or
hallmark > 0
```

And we will also add a column indicating that they are virsorter2 positive.

In [ ]:
vs2_keeps = vs2df[(vs2df['max_score'] > 0.9) & 
((vs2df['max_score_group'] == 'ssDNA') | (vs2df['hallmark'] > 0))].copy()

vs2_keeps['vs2_pos'] = 1

### VIBRANT

Let's check out the directory contents:

In [ ]:
!ls {vib_outdir}/*/

In [ ]:
!ls {vib_outdir}/*/VIBRANT_*

In [ ]:
vibdf = pd.read_csv(os.path.join(vib_outdir, 
                                 'VIBRANT_{}_contigs'.format(sag),
                                 'VIBRANT_results_{}_contigs'.format(sag),
                                 'VIBRANT_genome_quality_{}_contigs.tsv'.format(sag)),
                   sep = "\t")

In [ ]:
vibdf

We're going to keep all of these hits for now, let's just add some columns for future merging:

In [ ]:
vibdf['sag'] = [i.split("_")[0] for i in vibdf['scaffold']]
vibdf['contig'] = ["_".join(i.split("_")[:-2]) if 'fragment' in i else i for i in vibdf['scaffold']]
vibdf['vib_pos'] = 1
vibdf['vib_type'] = ['prophage' if 'fragment' in i else 'complete contig' for i in vibdf['scaffold']]

### DeepVirFinder

And now let's check out the deep vir finder results

In [ ]:
!ls {dvf_outdir}

Only one output table!  Let's check it out:

In [ ]:
dvfdf = pd.read_csv(os.path.join(dvf_outdir, '{}_contigs.fasta_gt1bp_dvfpred.txt'.format(sag)), sep = "\t")

In [ ]:
dvfdf['sag'] = [i.split("_")[0] for i in dvfdf['name']]
dvfdf['contig'] = dvfdf['name']


The thresholds we are going to apply to these outputs are:  

dvf_score_min = 0.9   
dvf_pvalue_max = 0.05

In [ ]:
dvf_keep = dvfdf[(dvfdf['score'] > 0.9) & (dvfdf['pvalue'] < 0.05)].copy()
dvf_keep['dvf_pos'] = 1

Now let's merge these table together, and see what our SAGs look like:

In [ ]:

# defining a smaller number of columns to keep
keep_cols = ['contig','sag','vs2_pos','vs2_type','vib_pos','vib_type','dvf_pos']

# merging the virus finding outputs, and keeping only the above columns
merged_df = dvf_keep.merge(vs2_keeps, how = 'outer').merge(vibdf, how = 'outer')[keep_cols]

# replacing 'na' values with 0 for the virus finder results columns
merged_df[['vs2_pos','vib_pos','dvf_pos']] = merged_df[['vs2_pos','vib_pos','dvf_pos']].fillna(0)

# creating a consensus score per contig based on how many virus finders identified it.
merged_df['consensus_score'] = merged_df['vs2_pos'] + merged_df['vib_pos'] + merged_df['dvf_pos']
merged_df['consensus_score'].value_counts()

In [ ]:
merged_df.groupby(['dvf_pos','vib_pos','vs2_pos'], as_index = False)['contig'].count()

Finally, write this final dataframe to my working directory to reference later:

In [ ]:
merged_df.to_csv(os.path.join(sagdir, '{}_vfinding_vcandidates_merged_table.csv'.format(sag)), index = False)

Questions:

Which virus finder identified the most contigs?  
How much overlap is there between virus candidate contigs identified by different tools?